In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

#Load and unzip iNaturalist zip file onto server, then remove zip to optimize performance
zip_path = "drive/MyDrive/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip
print("Extracted the dataset");

Mounted at /content/drive
Extracted the dataset


In [ ]:
!pip install wandb -qU
import wandb
wandb.login(key='2b25045507d6a89b66edf89be892f3687346ed10')

wandb: Currently logged in as: ee21s125. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
from glob import glob
import torch
import torchvision.transforms as transforms
import numpy as np
import time
from torch.optim.lr_scheduler import CosineAnnealingLR



# Dataset Class for Setting up the data loading process
class PreProcessDataset(Dataset):
    def __init__(self, root_dir, image_size, mode = 'train', augument = 'True'):
        self.root_dir = root_dir;
        self.mode = mode;
        self.augument = augument;
        self.size = image_size;
        self.prepareDataset();
        self.imageTransform();

    def prepareDataset(self):
        self.files = [];
        self.labels = [];
        self.classes = [];

        directory_path = os.path.join(self.root_dir, 'train');
        folders = sorted(os.listdir(directory_path));

        for i in folders:
              if i[0]!= ".":
                self.classes.append(i);
        print("Classes:",self.classes);

        if self.mode == 'train':
            for i in range(len(self.classes)):
                files = glob(os.path.join(directory_path, self.classes[i], '*.jpg'));
                self.labels += [i]*len(files);
                self.files += files;

        elif self.mode == 'test':
            directory_path = os.path.join(self.root_dir, 'val');
            for i in range(len(self.classes)):
                files = glob(os.path.join(directory_path, self.classes[i], '*.jpg'));
                self.labels += [i]*len(files)
                self.files += files
        else:
            print("Invalid Mode");
            return None

    def imageTransform(self):
      if (self.augument):
        self.transform = transforms.Compose([
            transforms.RandomRotation(30),
            #transforms.RandomResizedCrop(self.size),
            transforms.CenterCrop(226),
            transforms.RandomHorizontalFlip(),
            transforms.Resize((256,256)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]);
      else:
        self.transform = transforms.Compose([transforms.RandomResizedCrop(self.size),transforms.ToTensor()]);

        # self.transforms = transforms.Compose([
        #     transforms.RandomRotation(30),
        #     transforms.RandomResizedCrop((256,256)),
        #     transforms.RandomHorizontalFlip(),
        #     transforms.ToTensor(),
        #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #      transforms.Normalize([0.2356, 0.2300, 0.1948], [0.1096, 0.1055, 0.1075])
        # ]);

    def __getitem__(self, index):
        img = Image.open(self.files[index]).convert('RGB');
        img = self.transform(img);
        label = self.labels[index];
        label = torch.tensor(label, dtype = torch.long);
        return img, label;

    def __len__(self):
        return len(self.files);

class convolutionNN(torch.nn.Module):
    def __init__(self, image_size, num_input, num_output, filter_organization, batch_norm, num_layers, num_filters, filter_size, maxpool_size, dropout, activation, num_dense_neurons):
        super(convolutionNN,self).__init__();
        self.image_size = image_size;
        self.num_input = num_input;
        self.num_classes = num_output;
        self.filter_organization = filter_organization;
        self.batch_norm = batch_norm;
        self.num_layers = num_layers;
        self.num_filters = num_filters;
        self.filter_size = filter_size;
        self.maxpool_size = maxpool_size;
        self.dropout = dropout;
        self.num_dense_neurons = num_dense_neurons;

        if activation == 'ReLU':
          self.activation = torch.nn.ReLU;
        if activation == 'GELU':
          self.activation = torch.nn.GELU;
        if activation == "SiLU":
          self.activation = torch.nn.SiLU;
        if activation == "Mish":
          self.activation = torch.nn.Mish;
        if activation == 'LeakyReLU':
          self.activation = torch.nn.LeakyReLU;

        self.filter_padding = 1;
        self.filter_stride = 1;
        self.maxpool_stride = 2;
        self.layers = torch.nn.ModuleList();
        num_filters_used = [self.num_filters];


        for i in range(self.num_layers):
          #Since filter sizes are reducing significantly
          self.filter_padding = (self.filter_size-1)//2;
          self.convolutionBlock();
          self.num_input = self.num_filters;
          if(self.filter_organization == 'half'):
            self.num_filters = self.num_filters//2;
            num_filters_used.append(self.num_filters);
          if(self.filter_organization == 'same'):
            self.num_filters = self.num_filters;
            num_filters_used.append(self.num_filters);
          if(self.filter_organization == 'double'):
            self.num_filters = self.num_filters*2;
            num_filters_used.append(self.num_filters);

        final_dim = self.computeInputSizeToDense();
        conv_output_dim = final_dim * final_dim * num_filters_used[-2];
        print(final_dim, conv_output_dim ,num_filters_used);


        self.FCN = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(conv_output_dim, self.num_dense_neurons),
            self.activation(),
            torch.nn.Dropout(self.dropout),
            torch.nn.Linear(self.num_dense_neurons, self.num_classes)
            );

    def convolutionBlock(self):
        self.layers.append(torch.nn.Conv2d(self.num_input,self.num_filters, self.filter_size, padding = self.filter_padding));
        self.layers.append(self.activation());
        self.layers.append(torch.nn.MaxPool2d(self.maxpool_size, stride = self.maxpool_stride));
        if(self.batch_norm == True):
          self.layers.append(torch.nn.BatchNorm2d(self.num_filters));

    def computeInputSizeToDense(self):
        input = self.image_size;
        #print("input:",input)
        for i in range(self.num_layers):
          #print("Padding:", self.filter_padding);
          conv_output = np.ceil(float(input - self.filter_size + (2*self.filter_padding))/self.filter_stride) + 1;
          #print("Conv output Dim:",conv_output);
          maxpool_output = np.floor(float(conv_output - self.maxpool_size)/self.maxpool_stride) + 1;
          #print("maxpool_output:",maxpool_output);
          input = maxpool_output;
        return int(maxpool_output);

    def forward(self,x):
      #print("Input:", self.image_size, "x.size():", x.size())
      for layer in self.layers:
        x = layer(x);
        #print("layer: ", layer, "x.size():", x.size())
      x = self.FCN(x);
      return torch.nn.functional.softmax(x,dim=1);



def train(model, dataset, optimizer, criterion, device):

    epoch_loss = 0
    epoch_correct_predictions = 0
    epoch_total_samples = 0
    model.train();
    for i, (images,labels) in enumerate(dataset):
        #setting gradients to zero again to prevent any incorrect calculation
        optimizer.zero_grad();
        #loading the data accordingly
        #images, labels = next(iter(dataset));
        images, labels = images.to(device), labels.to(device);
        #forward pass
        predicted = model.forward(images);
        #calculate loss
        loss = criterion(predicted,labels);
        #gradient descent
        loss.backward();
        #optimizer updates the parameters
        optimizer.step();
        #print("Lenth labels:",len(labels));
        _, predicted_labels = torch.max(predicted, 1);
        correct_predictions = (predicted_labels == labels).sum().item();
        batch_accuracy = correct_predictions / len(labels);

        # Accumulate loss and accuracy for the epoch
        epoch_loss += loss.item() * len(labels);
        epoch_correct_predictions += correct_predictions;
        epoch_total_samples += len(labels);

    # Calculate average loss and accuracy for the epoch
    epoch_loss /= epoch_total_samples;
    epoch_accuracy = epoch_correct_predictions / epoch_total_samples;

    return epoch_loss, epoch_accuracy;


def validate(model, dataset, criterion, device):

    epoch_loss = 0
    epoch_correct_predictions = 0
    epoch_total_samples = 0
    model.train();
    for i, (images,labels) in enumerate(dataset):
        #loading the data accordingly
        #images, labels = next(iter(dataset));
        images, labels = images.to(device), labels.to(device);
        #forward pass
        predicted = model.forward(images);
        #calculate loss
        loss = criterion(predicted,labels);
        #print("Lenth labels:",len(labels));

        _, predicted_labels = torch.max(predicted, 1);
        correct_predictions = (predicted_labels == labels).sum().item();
        batch_accuracy = correct_predictions / len(labels);

        # Accumulate loss and accuracy for the epoch
        epoch_loss += loss.item() * len(labels);
        epoch_correct_predictions += correct_predictions;
        epoch_total_samples += len(labels);

    # Calculate average loss and accuracy for the epoch
    epoch_loss /= epoch_total_samples;
    epoch_accuracy = epoch_correct_predictions / epoch_total_samples;

    return epoch_loss, epoch_accuracy;




In [ ]:
def main():
    wandb.init(project='EE21S125_DL_A2')
    #Preprocessing Parameters
    augument = wandb.config.data_augmentation;
    num_dense_neurons = wandb.config.num_dense_neurons;
    learning_rate = wandb.config.learning_rate;
    dropout = wandb.config.dropout;
    activation = wandb.config.activation;
    num_filters = wandb.config.num_filters;
    filter_organization = wandb.config.filter_organization;
    batch_norm = wandb.config.batch_normalization;
    num_epochs = wandb.config.num_epochs;
    weight_decay = wandb.config.weight_decay;
    optimizer = wandb.config.optimizer;

    # augument = False;
    # num_dense_neurons = 256;
    # learning_rate = 0.001;
    # dropout = 0.3;
    # activation = 'ReLU';
    # num_filters = 32;
    # filter_organization = 'double';
    # batch_norm = True;
    # num_epochs = 5;
    # weight_decay = 0;
    # optimizer = 'NAdam'


    dataset_directory = 'inaturalist_12K';
    image_size = 256;
    batch_size = 10;
    num_input = 3;
    num_output = 10;
    num_layers = 5;
    filter_size = 3;
    maxpool_size = 2;
    mode = 'train';

    wandb.run.name = ("Aug_"+str(augument)+
                      "_bn_"+str(batch_norm)+
                      "_fcn_"+str(num_dense_neurons)+
                      "_do_"+str(dropout)+
                      "_act_"+str(activation)+
                      "_nf_"+str(num_filters)+
                      "_fo_"+str(filter_organization)+
                      "_lr_"+str(learning_rate)
                      );

    training_set = PreProcessDataset(dataset_directory, image_size, mode, augument);
    train_size = int(0.8*len(training_set));
    validate_size = len(training_set) - train_size;
    train_dataset, validate_dataset = torch.utils.data.random_split(training_set, [train_size, validate_size]);
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0);
    validate_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0);

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu');
    print("Device:", device);

    # Creating an instance of the model
    model = convolutionNN(image_size, num_input, num_output, filter_organization, batch_norm, num_layers, num_filters, filter_size, maxpool_size, dropout, activation, num_dense_neurons).to(device);

    criterion = torch.nn.CrossEntropyLoss();
    if optimizer == 'NAdam':
      optimizer=torch.optim.NAdam(model.parameters(),lr=learning_rate,weight_decay=weight_decay);
      anneal = CosineAnnealingLR(optimizer, T_max=num_epochs/2, eta_min = 0.0001);
    elif optimizer == 'SGD':
      optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate,weight_decay=weight_decay);
      anneal = CosineAnnealingLR(optimizer, T_max=num_epochs/2, eta_min = 0.0001)

    for epoch in range(num_epochs):
      start_time = time.time();
      train_loss,train_accuracy = train(model,train_loader,optimizer,criterion,device);
      validate_loss,validate_accuracy = validate(model,validate_loader,criterion,device);
      anneal.step();
      end_time = time.time();
      elapsed_time = end_time - start_time;

      print("Epoch Time Taken:", elapsed_time);
      print("Training Accuracy:", train_accuracy," Training Loss:", train_loss, " Validation Accuracy:", validate_accuracy, " Validate Loss:", validate_loss," Epoch:", epoch +1);
      wandb.log({'train_loss': train_loss, 'train_accuracy':train_accuracy, 'validate_loss': validate_loss, 'validate_accuracy': validate_accuracy, 'epoch':epoch + 1})
    wandb.finish();
    print("TRAINING AND VALIDATION COMPLETE");




### Used for checking the transform value ####
    # trainloader = DataLoader(trainset, batch_size=2, shuffle=True, num_workers=0);
    # print("Trainloader:", trainloader);
    # #data_loader = torch.utils.data.DataLoader(trainset.__getitem__(), batch_size=100, shuffle=False)
    # mean = torch.zeros(3)
    # std = torch.zeros(3)
    # num_samples = 0

    # for inputs, _ in trainloader:
    #     batch_size = inputs.size(0)
    #     num_samples += batch_size
    #     # Compute mean across batch and channels
    #     mean += torch.mean(inputs, dim=(0, 2, 3))
    #     # Compute std across batch and channels
    #     std += torch.std(inputs, dim=(0, 2, 3))

    # mean /= num_samples
    # std /= num_samples

    # print("Mean:", mean)
    # print("Standard Deviation:", std)



if __name__ == "__main__":
    sweep_config = {
                    'method': 'random',  # Choose 'grid', 'random', 'bayes', etc.
                    'metric': {
                                'name': 'validate_accuracy',
                                'goal': 'maximize'
                              },
                    'parameters': {
                        'num_filters': {'values': [32, 64, 128]},
                        'activation': {'values': ['ReLU', 'GELU', 'SiLU', 'Mish','LeakyReLU']},
                        'filter_organization': {'values': ['same', 'double', 'half']},
                        'data_augmentation': {'values': [True, False]},
                        'batch_normalization': {'values': [True, False]},
                        'num_dense_neurons':{'values': [128,256]},
                        'dropout': {'values': [0.2, 0.3]},
                        'learning_rate': {'values': [1e-4,1e-5]},
                        'weight_decay': {'values': [0, 0.0005, 0.5]},
                        'optimizer': {'values': ['NAdam','SGD']},
                        'num_epochs': {'values': [15]}

                    }
                  }
    sweep_id = wandb.sweep(sweep= sweep_config, project='EE21S125_DL_A2');
    wandb.agent(sweep_id, function = main,count=25);
    #main();


Create sweep with ID: ofmdawlk
Sweep URL: https://wandb.ai/ee21s125/EE21S125_DL_A2/sweeps/ofmdawlk


wandb: Agent Starting Run: eucr52zs with config:
wandb: 	activation: Mish
wandb: 	batch_normalization: True
wandb: 	data_augmentation: False
wandb: 	dropout: 0.3
wandb: 	filter_organization: half
wandb: 	learning_rate: 1e-05
wandb: 	num_dense_neurons: 256
wandb: 	num_epochs: 15
wandb: 	num_filters: 128
wandb: 	optimizer: NAdam
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Classes: ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Device: cuda
input: 256
8 512 [128, 64, 32, 16, 8, 4]
Epoch Time Taken: 233.1797194480896
Training Accuracy: 0.1532691586448306  Training Loss: 2.2815413496496975  Validation Accuracy: 0.17527190898862358  Validate Loss: 2.2675692230958555  Epoch: 1
